# Semantyczne Jądro z integracją serwera OpenBnB MCP

Ten notatnik pokazuje, jak używać Semantycznego Jądra z rzeczywistym serwerem OpenBnB MCP do wyszukiwania prawdziwych noclegów Airbnb za pomocą MCPStdioPlugin. Do dostępu do LLM wykorzystuje Azure AI Foundry. Aby skonfigurować zmienne środowiskowe, możesz postępować zgodnie z [Lekcją konfiguracji](/00-course-setup/README.md)


## Zaimportuj potrzebne pakiety


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Tworzenie połączenia wtyczki MCP

Połączymy się z [serwerem MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) za pomocą MCPStdioPlugin. Ten serwer udostępnia funkcjonalność wyszukiwania Airbnb za pośrednictwem pakietu @openbnb/mcp-server-airbnb.


## Tworzenie klienta

W tym przykładzie użyjemy Azure AI Foundry do uzyskania dostępu do LLM. Upewnij się, że Twoje zmienne środowiskowe są poprawnie skonfigurowane.


## Konfiguracja środowiska

Skonfiguruj ustawienia Azure OpenAI. Upewnij się, że masz ustawione następujące zmienne środowiskowe:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Zrozumienie integracji OpenBnB MCP

Ten notebook łączy się z **prawdziwym serwerem OpenBnB MCP**, który oferuje rzeczywiste funkcje wyszukiwania Airbnb.

### Jak to działa:

1. **MCPStdioPlugin**: Korzysta z komunikacji za pomocą standardowego wejścia/wyjścia z serwerem MCP
2. **Prawdziwy pakiet NPM**: Pobiera i uruchamia `@openbnb/mcp-server-airbnb` za pomocą npx
3. **Dane na żywo**: Zwraca rzeczywiste dane o nieruchomościach Airbnb z ich API
4. **Odkrywanie funkcji**: Agent automatycznie odkrywa dostępne funkcje serwera MCP

### Dostępne funkcje:

Serwer OpenBnB MCP zazwyczaj oferuje:
- **search_listings** - Wyszukiwanie nieruchomości Airbnb według lokalizacji i kryteriów
- **get_listing_details** - Pobieranie szczegółowych informacji o konkretnych nieruchomościach
- **check_availability** - Sprawdzanie dostępności w określonych terminach
- **get_reviews** - Pobieranie opinii o nieruchomościach
- **get_host_info** - Pobieranie informacji o gospodarzach nieruchomości

### Wymagania wstępne:

- Zainstalowany **Node.js** na Twoim systemie
- **Połączenie z Internetem** w celu pobrania pakietu serwera MCP
- Dostępny **NPX** (dołączony do Node.js)

### Testowanie połączenia:

Możesz ręcznie przetestować serwer MCP, uruchamiając:
```bash
npx -y @openbnb/mcp-server-airbnb
```

To pobierze i uruchomi serwer OpenBnB MCP, z którym Semantic Kernel następnie połączy się, aby uzyskać rzeczywiste dane Airbnb.


## Uruchamianie Agenta z serwerem OpenBnB MCP

Teraz uruchomimy Agenta AI, który łączy się z serwerem OpenBnB MCP, aby wyszukać prawdziwe zakwaterowanie Airbnb w Sztokholmie dla 2 dorosłych i 1 dziecka. Możesz swobodnie zmienić listę `user_inputs`, aby dostosować kryteria wyszukiwania.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Podsumowanie
Gratulacje! Udało Ci się stworzyć agenta AI, który integruje się z wyszukiwaniem zakwaterowania w rzeczywistym świecie za pomocą protokołu Model Context Protocol (MCP):

## Wykorzystane technologie:
- Semantic Kernel - Do budowy inteligentnych agentów z Azure OpenAI
- Azure AI Foundry - Do obsługi modeli językowych i uzupełniania rozmów
- MCP (Model Context Protocol) - Do standaryzowanej integracji narzędzi
- OpenBnB MCP Server - Do rzeczywistego wyszukiwania zakwaterowania na Airbnb
- Node.js/NPX - Do uruchamiania zewnętrznego serwera MCP

## Czego się nauczyłeś:
- Integracja MCP: Łączenie agentów Semantic Kernel z zewnętrznymi serwerami MCP
- Dostęp do danych w czasie rzeczywistym: Wyszukiwanie rzeczywistych ofert Airbnb za pomocą API
- Komunikacja protokołu: Korzystanie z komunikacji stdio między agentem a serwerem MCP
- Odkrywanie funkcji: Automatyczne wykrywanie dostępnych funkcji z serwerów MCP
- Strumieniowe odpowiedzi: Rejestrowanie i logowanie wywołań funkcji w czasie rzeczywistym
- Renderowanie HTML: Formatowanie odpowiedzi agenta w postaci stylizowanych tabel i interaktywnych wyświetleń

## Kolejne kroki:
- Zintegruj dodatkowe serwery MCP (pogoda, loty, restauracje)
- Zbuduj system wieloagentowy łączący protokoły MCP i A2A
- Stwórz własne serwery MCP dla swoich źródeł danych
- Zaimplementuj trwałą pamięć rozmów między sesjami
- Wdróż agenta w Azure Functions z orkiestracją serwera MCP
- Dodaj uwierzytelnianie użytkowników i funkcje rezerwacji



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
